In [23]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import time
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from hyperopt.pyll.base import scope
from xgboost import XGBClassifier

In [24]:
data = pd.read_csv('diabetes.csv')
print(data)

     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50        1  
1                  

In [25]:
X = data.drop('Outcome', axis=1)
y = data['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [26]:
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=3, objective='binary:logistic', reg_alpha=30, reg_lambda=80, booster='dart')
start = time.time()
bst.fit(X_train, y_train)
end = time.time()

y_pred = bst.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"Точность: {acc}")
print(f"Время обучения: {end - start} секунд")

Точность: 0.7532467532467533
Время обучения: 0.004999399185180664 секунд


In [55]:
#Подбор гиперпараметров с помощью Random Search
print("Random Search:")
param_dist = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(1, 20),
    'max_features': ['sqrt', 'log2', None] + list(range(1, X.shape[1]+1)),
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 11)
}

rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(
    rf, 
    param_distributions=param_dist,
    n_iter=50,
    cv=5,
    scoring='f1',
    random_state=42,
    n_jobs=-1
)

start_time1 = time.time()
random_search.fit(X_train, y_train)
end_time1 = time.time()

print("Лучшие параметры:", random_search.best_params_)
print("Лучший F1-score:", random_search.best_score_)
print("Время выполнения Random Search:", end_time1 - start_time1, "секунд")

Random Search:
Лучшие параметры: {'max_depth': 9, 'max_features': 5, 'min_samples_leaf': 1, 'min_samples_split': 9, 'n_estimators': 112}
Лучший F1-score: 0.6556010199845816
Время выполнения Random Search: 4.70731782913208 секунд


In [56]:
# Оценка на тестовых данных
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test)
print("F1-score на тестовых данных:", f1_score(y_test, y_pred))
print("Accuracy на тестовых данных:", accuracy_score(y_test, y_pred))

F1-score на тестовых данных: 0.6545454545454545
Accuracy на тестовых данных: 0.7532467532467533


In [57]:
#Подбор гиперпараметров с помощью TPE (Hyperopt)
max_features_options = ['sqrt', 'log2'] + list(range(1, X.shape[1]+1))

def objective(params):
    try:
        rf = RandomForestClassifier(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            max_features=params['max_features'],
            min_samples_split=int(params['min_samples_split']),
            min_samples_leaf=int(params['min_samples_leaf']),
            random_state=42,
            n_jobs=-1
        )
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
        score = f1_score(y_test, y_pred)
        return {'loss': -score, 'status': STATUS_OK}
    except Exception as e:
        return {'loss': 0, 'status': STATUS_OK, 'exception': str(e)}

In [58]:
space = {
    'n_estimators': scope.int(hp.quniform('n_estimators', 50, 200, 1)),
    'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
    'max_features': hp.choice('max_features', max_features_options),
    'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 10, 1))
}

In [59]:
print("TPE (Hyperopt):")
trials = Trials()
start_time2 = time.time()

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    rstate=np.random.default_rng(42)
)

end_time2 = time.time()
best_params = {
    'n_estimators': int(best['n_estimators']),
    'max_depth': int(best['max_depth']),
    'max_features': max_features_options[best['max_features']],
    'min_samples_split': int(best['min_samples_split']),
    'min_samples_leaf': int(best['min_samples_leaf'])
}

print("Лучшие параметры:", best_params)
print("Лучший F1-score:", -trials.best_trial['result']['loss'])
print("Accuracy на тестовых данных:", accuracy_score(y_test, y_pred))
print("Время выполнения TPE:", end_time2 - start_time2, "секунд")

TPE (Hyperopt):
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:09<00:00,  5.48trial/s, best loss: -0.6707317073170732]
Лучшие параметры: {'n_estimators': 155, 'max_depth': 8, 'max_features': 'log2', 'min_samples_split': 10, 'min_samples_leaf': 3}
Лучший F1-score: 0.6707317073170732
Accuracy на тестовых данных: 0.7532467532467533
Время выполнения TPE: 9.121214866638184 секунд


In [60]:
#Сравнение результатов
print("\n3. Сравнение результатов:")
print("Random Search:")
print(f"- Время: {end_time1 - start_time1:.2f} сек")
print(f"- F1-score: {random_search.best_score_:.4f}")
print(f"- Accuracy: {accuracy_score(y_test, random_search.best_estimator_.predict(X_test)):.4f}")

print("\nTPE (Hyperopt):")
print(f"- Время: {end_time2 - start_time2:.2f} сек")
print(f"- F1-score: {-trials.best_trial['result']['loss']:.4f}")
print(f"- Accuracy: {accuracy_score(y_test, best_rf_tpe.predict(X_test)):.4f}")


3. Сравнение результатов:
Random Search:
- Время: 4.71 сек
- F1-score: 0.6556
- Accuracy: 0.7532

TPE (Hyperopt):
- Время: 9.12 сек
- F1-score: 0.6707
- Accuracy: 0.7662


In [ ]:
Вывод:
Оба метода нашли схожие оптимальные параметры
С помощью Random Search мы получили хорошую точность за короткое время, но у TPE результаты все таки лучше.
Для данной задачи оба метода показали хорошие результаты, но TPE может быть предпочтительнее для более сложных случаев, в то время как Random Search является хорошим базовым подходом для подбора гиперпараметров.